#### **0. CHUẨN BỊ**

##### **0.1. CẤU HÌNH CÁC THAM SỐ**

In [1]:
import re
import sys
sys.path.append('../')
import os
import pandas as pd
from collections import Counter, defaultdict
import datetime
from tqdm import tqdm
import string
from copy import deepcopy

import hashlib

pd.set_option('display.max_rows', None)

SETTING_PARAMS = {
    'Apache': {
        'log_file': './logs/Apache/Apache_full.log',
        'log_template': './logs/Apache/Apache_full.log_templates.csv',
        'log_structure': './logs/Apache/Apache_full.log_structured.csv',
        'log_format': '\[<Time>\] \[<Level>\] <Content>',
        'filters': [],
        'regexs': [
            r'\/(?:\w+\/){2,}\w+\.\w+$',
            r'\/(?:[^\/\s]+\/)*[^\/\s]*'
        ],    
    },
    'HealthApp':{
        'log_file': './logs/HealthApp/HealthApp_full.log',
        'log_template': './logs/HealthApp/HealthApp_full.log_templates.csv',
        'log_structure': './logs/HealthApp/HealthApp_full.log_structured.csv',
        'log_format': '<Time>\|<Component>\|<Pid>\|<Content>',
        'filters': [],
        'regexs': [],  
    },
    'Mac': {
        'log_file': './logs/Mac/Mac_full.log',
        'log_template': './logs/Mac/Mac_full.log_templates.csv',
        'log_structure': './logs/Mac/Mac_full.log_structured.csv',
        'log_format': '<Month>  <Date> <Time> <User> <Component>\[<PID>\]( \(<Address>\))?: <Content>',
        'filters': [],
        'regexs': [
            r'([\w-]+\.){2,}[\w-]+',
            r'https?:\/\/(?:[^\/\s]+\/?)*',
            r'\S*\/(?:[^\/\s]+\/){1,}[^\/\s]*'
        ],  
    },
    'Linux': {
        'log_file': './logs/Linux/Linux_full.log',
        'log_template': './logs/Linux/Linux_full.log_templates.csv',
        'log_structure': './logs/Linux/Linux_full.log_structured.csv',
        'log_format': '<Month> <Date> <Time> <Level> <Component>(\[<PID>\])?: <Content>',
        'filters': [],
        'regexs': [r'(\d+\.){3}\d+', r'\d{2}:\d{2}:\d{2}'],  
    },
    'OpenSSH': {
        'log_file': './logs/OpenSSH/OpenSSH_full.log',
        'log_template': './logs/OpenSSH/OpenSSH_full.log_templates.csv',
        'log_structure': './logs/OpenSSH/OpenSSH_full.log_structured.csv',
        'log_format': '<Date> <Day> <Time> <Component> sshd\[<Pid>\]: <Content>',
        'filters': [],
        'regexs': [r"(\d+):"],    
    },
}

##### **0.2. CLASS**

In [2]:
class LogCluster:
    def __init__(self, keyGroup, logTemplate, tokens, length, logIDL=None, dynamic_tokenL=None, static_tokenL=None):
        self.keyGroup = keyGroup
        self.logTemplate = logTemplate
        self.tokens = tokens
        self.length = length
        self.logIDL = logIDL if logIDL is not None else []
        self.dynamic_tokenL = dynamic_tokenL if dynamic_tokenL is not None else {}
        self.static_tokenL = static_tokenL if static_tokenL is not None else {}
        
    def __str__(self):
        dynamic_str = "Dynamic Tokens: {\n"
        for k, v in self.dynamic_tokenL.items():
            dynamic_str += f"  {k}: {v},\n"
        dynamic_str += "}"
        
        static_str = "Cover Tokens: {\n"
        for k, v in self.static_tokenL.items():
            static_str += f"  {k}: {v},\n"
        static_str += "}"
        return (
            f"Key: {self.keyGroup}\n"
            f"Template: {self.logTemplate}\n"
            f"Tokens: {self.tokens}\n"
            f"Length: {self.length}\n"
            f"Len LogIDs: {len(self.logIDL)}\n"
            f"{dynamic_str}\n"
            f"{static_str}\n"
        )

#### **1. Các phương thức sử dụng**

##### **1.1. Read data**
- **`log_to_dataframe()`**

- **`generate_logformat_regex()`**

- **`load_data()`**

In [3]:
# =============================== READ DATA =============================== #
def log_to_dataframe(log_file, regex, headers):
    """ Phương thức chuyển đổi file log thành dataframe
    """ 
    log_messages = []
    linecount = 0
    with open(log_file, 'r', encoding="utf8") as fin:
        for line in fin.readlines():
            try:
                match = regex.search(line.strip())
                message = [match.group(header) for header in headers]
                log_messages.append(message)
                linecount += 1
            except Exception as e:
                pass
    logdf = pd.DataFrame(log_messages, columns=headers)
    logdf.insert(0, 'LineId', None)
    logdf['LineId'] = [i + 1 for i in range(linecount)]
    return logdf

def generate_logformat_regex(logformat):
    """ Phương thức tạo regex từ logformat, biểu thức định dạng của một event log: 
    Ex: 'log_format': '<Date> <Time> <Pid> <Level> <Component>: <Content>'
    """
    headers = []
    splitters = re.split(r'(<[^<>]+>)', logformat)
    regex = ''
    for k in range(len(splitters)):
        if k % 2 == 0:
            splitter = re.sub(' +', '\\\s+', splitters[k])
            regex += splitter
        else:
            header = splitters[k].strip('<').strip('>')
            regex += '(?P<%s>.*?)' % header
            headers.append(header)
    regex = re.compile('^' + regex + '$')
    return headers, regex

def load_data(logfile, logformat):
    """ Phương thức trả về một dataframe từ một file log chỉ định
    """
    log_headers, log_regex = generate_logformat_regex(logformat)
    logs_df = log_to_dataframe(logfile, log_regex, log_headers)
    return logs_df

##### **1.2. PRE_PROCESSING_0**
- **`pre0_hasNumbers()`**

- **`pre0_regexAndFilter()`**

- **`createSpecialRex()`**

In [4]:
# ========================== PRE_PROCESSING_0 ========================== #
def pre0_hasNumbers(s):
    return any(char.isdigit() for char in s)

def pre0_regexAndFilter(line, regexs = [], filters = []):
    for currentFil in filters:
        line = re.sub(currentFil, '', line)
    for currentRex in regexs:
        line = re.sub(currentRex, "<*>", line)
    return line.strip()

# ======================= CÁC PHƯƠNG THỨC BỔ TRỢ ======================= #
def createSpecialRex(dict_special_token):
    """ Tạo biểu thức chính quy cho các từ điển yêu cầu (không phân biệt hoa thường) """
    keywords = '|'.join([re.escape(k) for k in dict_special_token])
    return re.compile(f'({keywords})', flags=re.IGNORECASE)


##### **1.3. PHƯƠNG THỨC HỖ TRỢ NHÓM LOG**

- **`extractDynamicTok()`**

In [5]:
# def extractDynamicTok(groups, tokens):
#     dynamic_tokenL = defaultdict(list)
#     split_lines = [row['Content'].strip().split() for _, row in groups.iterrows()]

#     for pos, token in enumerate(tokens):
#         if "<*>" in token:
#             for line in split_lines:
#                 if pos < len(line):
#                     dynamic_tokenL[pos].append(line[pos])
#     return dict(dynamic_tokenL)

##### **1.4. PREPROCESSING_1**

- **`splitSpecialTok()`**

- **`mergeSpecialTok()`**

In [6]:
# ========================== PRE_PROCESSING_1 ========================== #
def splitSpecialTok(s, seps):
    """ Tách chuỗi s giữ lại ký tự phân tách trong seps. Trả về None nếu không có ký tự phân tách."""
    pattern = '|'.join(map(re.escape, seps))  
    if not re.search(pattern, s):             
        return None, None
    tokens = re.split(f'({pattern})', s)
    sep_token = [tok for tok in tokens if tok]   
    static_tokenL = ["<*>" if pre0_hasNumbers(tok) else tok for tok in sep_token]
    # static_str = mergeSpecialTok("".join(static_tokenL), seps)
        
    return sep_token, static_tokenL

# Phương thức gộp chuỗi
def mergeSpecialTok(token_str, seps):
    """ Nhận chuỗi string token đã xử lý và danh sách phân tách seps. Dùng regex để lặp lại việc thay thế mẫu: <*> + (kí tự phân tách giống nhau) + <*> => <*> hoặc <*>+... ==> <*>
    """
    sep_pattern = '|'.join(re.escape(sep) for sep in seps)

    prev = None
    while token_str != prev:
        prev = token_str
        
        # Gộp mẫu: <*> + (các ký tự phân tách giống nhau) + <*>
        token_str = re.sub(rf'(<\*>)(({sep_pattern})\3*)(<\*>)', r'<*>', token_str)
        
        # Gộp nhiều <*><*> liên tiếp:
        token_str = re.sub(r'(<\*>)+', r'<*>', token_str)

    return token_str

##**1.5. PHƯƠNG THỨC HỖ TRỢ NHÓM GROUP**##

- **class `MergeGroupTemplate`**

#### **1.5. MAIN FUNCTION**

- **`processLine()`**

In [7]:
# Viết lại phương thức xử lý theo từng dòng
def processLine(line, regexs, filters, pattern_special, punctuationL):
    """ Phương thức hỗ trợ xử lý từng dòng log """
    # ================== Xử lý regex và filter ================== #
    # tokens0 là list token sau khi tiền xử lý
    # groups_token là list tokens templates
    
    parsed = pre0_regexAndFilter(line['Content'], regexs=regexs, filters=filters)
    tokens0 = str(parsed).strip().split()

    groups_token = []         
    idx_dynamic_token = []
    # dynamic_token = []
    dynamic_tokenL = []
    static_tokenL = []
    # static_token = []
    
    for idx_tok, token in enumerate(tokens0):
        # Duyệt qua từng token, xử lý token có giá trị đặc biệt
        # Nếu token không chứa ký tự đặc biệt thì thêm vào groups_token, nếu có thì xử lý tiếp
        temp = pattern_special.sub('<*>', token)
        if not pre0_hasNumbers(temp): 
            groups_token.append(temp)
        else:
            groups_token.append("<*>")
            sep_token, static_tokL = splitSpecialTok(temp, punctuationL)
            if sep_token is not None:
                idx_dynamic_token.append(idx_tok)
                # dynamic_token.append(token)
                dynamic_tokenL.append(sep_token) 
                # static_token.append(sta_str)
                static_tokenL.append(static_tokL.copy())
    

    groupTem_str = f"{' '.join(groups_token)} : {len(groups_token)}"
    if idx_dynamic_token:
        posa = []
        for item in static_tokenL:
            posa.append(item[0])
        groupTem_str += f" : {' '.join([str(tok) for tok in idx_dynamic_token])} : {' '.join([str(len(item)) for item in static_tokenL])} : {' '.join(posa)}"
    
    return pd.Series({
        'GroupTemplate': hashlib.md5(groupTem_str.encode('utf-8')).hexdigest(),
        'GroupTokens': groups_token,
        'idxDynamicTok': idx_dynamic_token,
        # 'DynamicTok': dynamic_token, 
        'DynamicTokList': dynamic_tokenL,
        'StaticTokList': static_tokenL,
        # 'StaticTok': static_token,
        'TemplateProcessing': f""
    })

##### **1.6. PHƯƠNG THỨC KHÁC**

- **`write_df_to_txt()`**

- **`compare_templates()`**

In [8]:
def write_df_to_txt(df, filename):
    col_widths = [max(len(str(val)) for val in [col] + df[col].tolist()) for col in df.columns]

    with open(filename, 'w', encoding='utf-8') as f:
        header = '\t'.join(str(col).ljust(width) for col, width in zip(df.columns, col_widths))
        f.write(header + '\n')

        for row in df.itertuples(index=False):
            line = '\t'.join(str(val).ljust(width) for val, width in zip(row, col_widths))
            f.write(line + '\n')
            
def compare_templates(datasets, parse_df):
    structured_df = pd.read_csv(datasets['log_structure']) 
    unique_templates = structured_df['EventTemplate'].unique()
    print(f"SHAPE: {structured_df.shape}")
    print(f"SHAPE PARSER: {parse_df.shape}")
    print(f"Num of templates: {len(unique_templates)}")

    template_compare = {}
    for template in tqdm(unique_templates, total=len(parse_df), desc="Processing templates", unit="rows"):
        arr_index = structured_df[structured_df['EventTemplate'] == template].index.tolist()
        parse_template_series = parse_df.loc[arr_index, 'TemplateProcessing']
        parse_template_unique = parse_template_series.unique().tolist()

        hash_key = hash(template)
        template_compare[hash_key] = {
            'ground_truth': template,
            'parse': parse_template_unique,
            'index': arr_index,
            'length': len(template.strip().split()),
            'nums': len(arr_index)
        }
        
    sorted_items = sorted(
            template_compare.items(),
            key=lambda item: (item[1]['length'], item[1]['ground_truth'])
        )

    num_dif = 0
    for idx, (key, value) in enumerate(sorted_items, 1):
        if len(value['parse']) != 1 or value['parse'][0] != value['ground_truth']:
            num_dif += 1
            print(f"No. {idx}")
            print(f"Length: {value['length']}, Nums: {value['nums']}")
            print(f"Ground truth  : {value['ground_truth']}")
            print(f"Parse templs  : {value['parse']}")
            print(f"Length parse: {len(value['parse'])}")
            print("-" * 40)
    print(f"Total differences found: {num_dif}")

#### **2. KHUNG LÀM VIỆC CHÍNH**

In [9]:
# CẤU HÌNH CÁC THAM SỐ:
datasets = SETTING_PARAMS['Mac']
DICT_SPECIAL_TOKEN = ['true', 'false']
PUNCTUATIONL = '(),<>:;{}[]~='

N_MERGE = 2
ST = 0.6

logs_df = load_data(datasets['log_file'], datasets['log_format'])

# ================================ PROCESSING 0 ================================ #
parse_df = logs_df.copy()
parse_df['GroupTemplate'] = ""                                  # Lưu template sử dụng để nhóm
parse_df['GroupTokens'] = [[] for _ in range(len(parse_df))]    # Lưu list token của Group Teplate
parse_df['idxDynamicTok'] = [[] for _ in range(len(parse_df))]  # Lưu vị trí token động
# parse_df['DynamicTok'] = [[] for _ in range(len(parse_df))]     # Lưu token động theo vị trí tương ứng
parse_df['DynamicTokList'] = [[] for _ in range(len(parse_df))] # Lưu list token động theo vị trí tương ứng
# parse_df['StaticTok'] = [[] for _ in range(len(parse_df))]      # Lưu token tĩnh đã phân tích tương ứng
parse_df['StaticTokList'] = [[] for _ in range(len(parse_df))]  # Lưu list token tĩnh theo vị trí tương ứng
parse_df['TemplateProcessing'] = ""                             # Template cuối cùng sau khi xử lý


tqdm.pandas(desc="Tiền xử lý giai đoạn 0 và 1")
pattern_special = createSpecialRex(DICT_SPECIAL_TOKEN)          # Tạo ra các REGEX cho các giá trị đặc biệt
punctuationL = set(PUNCTUATIONL)                                # Lấy các giá trị phân tách đặc biệt và duy nhất

results = parse_df.progress_apply(
        lambda row: processLine(row, datasets['regexs'], datasets['filters'], pattern_special, punctuationL),
        axis=1
    )

for col in results.columns:
    parse_df[col] = results[col]

# compare_templates(datasets, parse_df)
# write_df_to_txt(parse_df, 'z_parse_df.txt')
# ==================================== END ==================================== #

Tiền xử lý giai đoạn 0 và 1: 100%|██████████| 100314/100314 [00:19<00:00, 5152.88it/s]


In [10]:
# ================================ GROUP LOG ================================ #
log_clusters_list = []                                          # List lưu trữ các nhóm log logCluster

unique_groups = parse_df.groupby("GroupTemplate")
print(len(unique_groups)) # in ra số nhóm chưa xử lý

for key, group_val in unique_groups:
    first_row = group_val.iloc[0]
    tokens = first_row['GroupTokens']
    
    if len(first_row["idxDynamicTok"]) != 0:                    # Ktra có token động chưa xử lý hay không?
        group_staticL = group_val['StaticTokList'].to_list()
        
        static_processingL = []                                 # List lưu các nhóm token động đã xử lý
        for idx in range(len(group_staticL[0])):
            cols_idx_gr = list(zip(*[x[idx] for x in group_staticL])) 
            static_idx = []
            for idx0, lst_idx in enumerate(cols_idx_gr):        # Lấy các phần tử theo cột của từng token đã được phân tách
                unique_idx = set(lst_idx)
                if len(unique_idx) > 1:                         # Vị trí có token khác nhau thì thành <*>
                    static_idx.append("<*>")
                else: 
                    static_idx.append(next(iter(unique_idx)))
            
            token_str = mergeSpecialTok("".join(static_idx), punctuationL)    
            
            static_processingL.append(token_str)                
    
        # Chuyển group thành các log cluster:
        for idx, val in enumerate(first_row["idxDynamicTok"]):
            tokens[val] = static_processingL[idx]
    
    logTemplate = " ".join(tokens)

    cluster = LogCluster(
        keyGroup= hashlib.md5(logTemplate.encode('utf-8')).hexdigest(),
        logTemplate=logTemplate,
        tokens=tokens,
        length=len(tokens),
        logIDL=group_val.index.tolist(),
        static_tokenL=None
    )
    log_clusters_list.append(cluster)
    # ==================================== END ==================================== #

845


In [ ]:
# ================================ MERGE TEMPLATE ================================ #
# Sử dụng ý tưởng giống như Drain, như sau:

# ===================== TẠO CLASS ===================== #
class MergeGroupTemplate:
    def __init__(self, st=0.6, n_merge=3, template_gr=None):
        self.ST = st
        self.N_MERGE = n_merge
        self.TEMPLATE_GR = template_gr if template_gr is not None else []
    
    def similarySeq(self, seq1, seq2):
        """ So sánh độ tương đồng giữa các token của 2 nhóm cluster dựa trên ý tưởng của Drain"""
        assert len(seq1) == len(seq2)
        simTokens = 0
        numOfPar = 0

        for token1, token2 in zip(seq1, seq2):
            if token1 == "<*>":
                numOfPar += 1
                continue
            if token1 == token2:
                simTokens += 1

        retVal = float(simTokens) / len(seq1)

        return retVal, numOfPar
    
    def fastMatchCLuster(self, seqGroupL, seq):
        choose_group = None
        maxSim = -1
        maxNumOfPara = -1
        maxGroup = None

        for gr in seqGroupL:
                curSim, curNumOfPara = self.similarySeq(gr[0].tokens, seq.tokens)
                if curSim > maxSim or (curSim == maxSim and curNumOfPara > maxNumOfPara):
                    maxSim = curSim
                    maxNumOfPara = curNumOfPara
                    maxGroup = gr
                    
                if maxSim >= ST:
                    choose_group = maxGroup
        return choose_group
    
    def generalizeGroup(self, group):
        """Tạo pattern chung bằng cách đếm số lượng token khác nhau tại mỗi vị trí"""
        mask_positions = set()
        tokensL = [s.tokens for s in group]
        
        for idx, col in enumerate(zip(*tokensL)):
            if len(set(col)) >= N_MERGE:
                mask_positions.add(idx)

        # Tạo pattern chung
        for seq in group:
            seq.tokens = [token if i not in mask_positions else "<*>" for i, token in enumerate(seq.tokens)]
            seq.logTemplate = " ".join(seq.tokens)

        # Gom nhóm lại theo pattern
        pattern_dict = defaultdict(list)
        for seq in group:
            key = tuple(seq.tokens)
            pattern_dict[key].append(seq)

        result = []
        for key, values in pattern_dict.items():
            if len(values) != 1: 
                logIDL = []
                for x in values:
                    logIDL.extend(x.logIDL)
                values[0].logIDL = logIDL
            result.append(values[0])
        return result
    
    def mergeGroup(self, printL=False):
        grouped_by_length = defaultdict(list)
        [grouped_by_length[t.length].append(t) for t in self.TEMPLATE_GR]
        
        newClusterGroupsL = []
        
        # Nhóm theo chiều dài:
        for length, groups_len in grouped_by_length.items():
            groupsSimTemL = []
            for log_clust in groups_len:
                matched_gr = self.fastMatchCLuster(groupsSimTemL, log_clust)
                if matched_gr is not None:
                    matched_gr.append(log_clust)
                else:
                    groupsSimTemL.append([log_clust])
            for group in groupsSimTemL:
                if len(group) == 1:
                    newClusterGroupsL.extend(group)
                else:
                    refined_groups = self.generalizeGroup(group)
                    newClusterGroupsL.extend(refined_groups)
        
        self.TEMPLATE_GR = newClusterGroupsL

        if printL:
            self.printList()
            
    def printList(self):
        print(len(self.TEMPLATE_GR))
        # df = pd.read_csv(datasets['log_template'])
        # print(len(df))

        sorted_list = sorted(self.TEMPLATE_GR, key=lambda log: (log.length, log.logTemplate))
        for e in sorted_list:
            print(f"{e.length:3} {e.logTemplate}")
# ===================== END CLASS ===================== #

merge_group = MergeGroupTemplate(st=ST, n_merge=N_MERGE, template_gr=log_clusters_list)
merge_group.mergeGroup(printL=True)


In [ ]:
for item in merge_group.TEMPLATE_GR:
    parse_df.loc[item.logIDL, "TemplateProcessing"] = item.logTemplate

compare_templates(datasets, parse_df)

In [13]:
# ================================ EXTEND DRAIN ================================ #
class Node:
    def __init__(self, childD=None, depth=0, digitOrtoken=None):
        if childD is None:
            childD = dict()
        self.childD = childD
        self.depth = depth
        self.digitOrtoken = digitOrtoken

class DrainTree:
    def __init__(self, depth=5, st=1, maxChild=20):
        self.depth = depth - 2
        self.st = st
        self.maxChild = maxChild
        self.rootNode = Node()

    
    def treeSearch(self, rn, seq):
        retLogClust = None

        seqLen = len(seq)
        if seqLen not in rn.childD:
            return retLogClust

        parentn = rn.childD[seqLen]

        currentDepth = 1
        for token in seq:
            if currentDepth >= self.depth or currentDepth > seqLen:
                break

            if token in parentn.childD:
                parentn = parentn.childD[token]
            elif "<*>" in parentn.childD:
                parentn = parentn.childD["<*>"]
            else:
                return retLogClust
            currentDepth += 1

        logClustL = parentn.childD

        retLogClust = self.fastMatch(logClustL, seq)

        return retLogClust

    def addSeqToPrefixTree(self, rn, logClust):
        seqLen = len(logClust.tokens)
        if seqLen not in rn.childD:
            firtLayerNode = Node(depth=1, digitOrtoken=seqLen)
            rn.childD[seqLen] = firtLayerNode
        else:
            firtLayerNode = rn.childD[seqLen]

        parentn = firtLayerNode

        currentDepth = 1
        for token in logClust.tokens:
            # Add current log cluster to the leaf node
            if currentDepth >= self.depth or currentDepth > seqLen:
                if len(parentn.childD) == 0:
                    parentn.childD = [logClust]
                else:
                    parentn.childD.append(logClust)
                break

            # If token not matched in this layer of existing tree.
            if token not in parentn.childD:
                if "<*>" in parentn.childD:
                    if len(parentn.childD) < self.maxChild:
                        newNode = Node(depth=currentDepth + 1, digitOrtoken=token)
                        parentn.childD[token] = newNode
                        parentn = newNode
                    else:
                        parentn = parentn.childD["<*>"]
                else:
                    if len(parentn.childD) + 1 < self.maxChild:
                        newNode = Node(depth=currentDepth + 1, digitOrtoken=token)
                        parentn.childD[token] = newNode
                        parentn = newNode
                    elif len(parentn.childD) + 1 == self.maxChild:
                        newNode = Node(depth=currentDepth + 1, digitOrtoken="<*>")
                        parentn.childD["<*>"] = newNode
                        parentn = newNode
                    else:
                        parentn = parentn.childD["<*>"]

            # If the token is matched
            else:
                parentn = parentn.childD[token]

            currentDepth += 1

    def seqDist(self, seq1, seq2):
        assert len(seq1) == len(seq2)
        simTokens = 0
        numOfPar = 0

        for token1, token2 in zip(seq1, seq2):
            if token1 == "<*>":
                numOfPar += 1
                continue
            if token1 == token2:
                simTokens += 1

        retVal = float(simTokens) / len(seq1)

        return retVal, numOfPar

    def fastMatch(self, logClustL, seq):
        retLogClust = None

        maxSim = -1
        maxNumOfPara = -1
        maxClust = None

        for logClust in logClustL:
            curSim, curNumOfPara = self.seqDist(logClust.tokens, seq)
            if curSim > maxSim or (curSim == maxSim and curNumOfPara > maxNumOfPara):
                maxSim = curSim
                maxNumOfPara = curNumOfPara
                maxClust = logClust

        if maxSim >= self.st:
            retLogClust = maxClust

        return retLogClust

    def printTree(self, node, dep):
        pStr = ""
        for i in range(dep):
            pStr += "\t"

        if node.depth == 0:
            pStr += "Root"
        elif node.depth == 1:
            pStr += "<" + str(node.digitOrtoken) + ">"
        else:
            pStr += node.digitOrtoken

        print(pStr)

        if node.depth == self.depth:
            return 1
        for child in node.childD:
            self.printTree(node.childD[child], dep + 1)

    def match_template(self, row):
        matchCluster = self.treeSearch(self.rootNode, row["GroupTokens"])
        return matchCluster.logTemplate if matchCluster else ""
    
    def parse(self, logs_df):
        # print("Parsing file: " + os.path.join(self.path, logName))
        start_time = datetime.now()
        logs_df['EventTemplate'] = ""             # Lưu kết quả xử lý

        tqdm.pandas(desc="So khớp ...")
        logs_df['EventTemplate'] = logs_df.progress_apply(self.match_template, axis=1)
        print("Parsing done. [Time taken: {!s}]".format(datetime.now() - start_time))
        return logs_df
    
    def createTree(self, newGroupL):
        self.rootNode = Node()
        for item in newGroupL:
            # matchCluster = self.treeSearch(self.rootNode, item.tokens)
            # if matchCluster is None:
            #     self.addSeqToPrefixTree(self.rootNode, item)
            # else: 
            #     matchCLuster.logIDL.extend(item.logIDL)
            
            self.addSeqToPrefixTree(self.rootNode, item)      # Chỉ cần dòng này là đủ
    
    
drain_tree = DrainTree(depth=5, st=1, maxChild=20)
drain_tree.createTree(merge_group.TEMPLATE_GR)


##### **@.2. HOÀN CHỈNH**

In [14]:
# def parse_DrainDS(datasets):
#     logs_df = load_data(datasets['log_file'], datasets['log_format'])
    
    
#     # =============================== CÁC CÂU LỆNH IN KIỂM TRA ===============================
#     logs_df.head(100)
    
    
    
# # =============== CHẠY CHƯƠNG TRÌNH ==============
# parse_DrainDS(SETTING_PARAMS['Mac'])